# Scoring an Anomaly Detection Dataset

This is a proposal to provide a score for an anomaly detection dataset to measure its difficulty when attempting to perform anomaly detection methods against it. This is largely based on the paper https://arxiv.org/pdf/1503.01158v2.pdf. 

We focus on two major properties namely:
1. Relative Frequency / Ratio of anomalies against data points in a dataset
2. Semantic variation  / clusterdness

In [1]:
# Import necessary libraries and path relative to project
from IPython.display import HTML, display
import pandas as pd
import numpy as np
import tabulate

In [2]:
# Setup the dataset

# Instantiate pandas DataFrame
data = pd.DataFrame()

# Chunk size for reading data
chunksize = 10000

# The reference to the dataset. Change this to 
dataset_file = '../data/creditcardfraud.csv'

print("Loading dataset '{}'...".format(dataset_file))

# Read each chunk and append to data frame
for i, chunk in enumerate(pd.read_csv(dataset_file, header=None, chunksize=chunksize)):
    print("Reading chunk %d" % (i + 1))
    data = data.append(chunk)

print("Done loading dataset...")
    
# Check for proper value of input dimensionality to be used by model
input_dim = len(data.columns) - 1
print("Input Dimensionality: %d" % (input_dim))

# Partition the data into positive_data and negative_data
positive_data = data[data[input_dim] == 1].iloc[:,:input_dim]
negative_data = data[data[input_dim] == -1].iloc[:,:input_dim]

Loading dataset '../data/creditcardfraud.csv'...
Reading chunk 1
Reading chunk 2
Reading chunk 3
Reading chunk 4
Reading chunk 5
Reading chunk 6
Reading chunk 7
Reading chunk 8
Reading chunk 9
Reading chunk 10
Reading chunk 11
Reading chunk 12
Reading chunk 13
Reading chunk 14
Reading chunk 15
Reading chunk 16
Reading chunk 17
Reading chunk 18
Reading chunk 19
Reading chunk 20
Reading chunk 21
Reading chunk 22
Reading chunk 23
Reading chunk 24
Reading chunk 25
Reading chunk 26
Reading chunk 27
Reading chunk 28
Reading chunk 29
Done loading dataset...
Input Dimensionality: 29


## Relative Frequency

This simply the ratio of number of anomalies in the entire dataset.

In [3]:
# Divide the number of anomalies vs the entire length of the dataset
# X: A pandas data frame
def score_relative_frequency(X):
    # Class column is always the last value
    idx_class = len(X.columns) - 1
    anomalies = X[X[idx_class] == -1]
    
    # Return the score in percentage format
    return (len(anomalies) / len(X)) * 100

print("Relative Frequency: %0.5f" % (score_relative_frequency(data)))

Relative Frequency: 0.17275


## Semantic Variation

A normalized clusterdness measure of given the following equation:

$$\log(\frac{\sigma^2_{n}}{\sigma^2_{a}})$$

where:

* $\sigma^2_{n}$ is the variance of normal data
* $\sigma^2_{a}$ is the variance of anomaly data

To deal with multi-dimensional data, we compute for the $\sigma^2$ by taking the covariance matrix of the data $X$ using the equation:

$$
\mathbf{Var}(X) = \mathbf{E}[(X - \mathbf{E}(X))(X - \mathbf{E}(X))^{T}]
\\
=
\begin{bmatrix}
\mathbf{Var}(X_1) & \cdots &
\mathbf{Cov}(X_1, X_p) \\
\vdots & \ddots & \vdots \\
\mathbf{Cov}{X_p, X_1} & \cdots &
\mathbf{Var}(X_p)
\end{bmatrix} 
\\
=
\frac{1}{n - 1}
\begin{bmatrix}
\sum_{i=1}^n(X_{i1} - \hat{X}_{1})^2 & \cdots &
\sum_{i=1}^n(X_{i1} - \hat{X}_{1})(X_{ip} - \hat{X}_{p}) \\
\vdots & \ddots & \vdots \\
\sum_{i=1}^n(X_{ip} - \hat{X}_{p})(X_{i1} - \hat{X}_{1})  & \cdots &
\sum_{i=1}^n(X_{ip} - \hat{X}_{p})^2
\end{bmatrix}
$$

We then take trace of the covariance matrix to give us the overall variance:

$$
\sigma^2 = \operatorname{tr}({\mathbf{Var}(X)})
$$

In [4]:
def score_semantic_variation(X):
    idx_class = len(X.columns) - 1
    
    # Partition the data into positive_data and negative_data
    positive_data = X[X[idx_class] == 1].iloc[:,:idx_class]
    negative_data = X[X[idx_class] == -1].iloc[:,:idx_class]
    
    var_n = np.trace(positive_data.cov().values)
    var_a = np.trace(negative_data.cov().values)
    
    return np.log(var_n / var_a)
    
print("Semantic variation: %0.5f" % (score_semantic_variation(data)))

Semantic variation: -1.81614


## Test against public datasets

In [5]:
datasets = [
    { "name": "annthyroid", "location": "https://happy-research.s3.ap-southeast-1.amazonaws.com/annthyroid.csv" },
    { "name": "backdoor", "location": "https://happy-research.s3.ap-southeast-1.amazonaws.com/backdoor.csv" },
    { "name": "bald", "location": "https://happy-research.s3.ap-southeast-1.amazonaws.com/bald.csv" },
    { "name": "bank", "location": "https://happy-research.s3.ap-southeast-1.amazonaws.com/bank.csv" },
    { "name": "cover", "location": "https://happy-research.s3.ap-southeast-1.amazonaws.com/cover.csv" },
    { "name": "creditcardfraud", "location": "https://happy-research.s3.ap-southeast-1.amazonaws.com/creditcardfraud.csv" },
    { "name": "donors", "location": "https://happy-research.s3.ap-southeast-1.amazonaws.com/donors.csv" },
    { "name": "kddcup99", "location": "https://happy-research.s3.ap-southeast-1.amazonaws.com/kddcup99.csv" },
    { "name": "magic04", "location": "https://happy-research.s3.ap-southeast-1.amazonaws.com/magic04.csv" },
    { "name": "mammography", "location": "https://happy-research.s3.ap-southeast-1.amazonaws.com/mammography.csv" },
    { "name": "musk", "location": "https://happy-research.s3.ap-southeast-1.amazonaws.com/musk.csv" },
    { "name": "pageblocks", "location": "https://happy-research.s3.ap-southeast-1.amazonaws.com/pageblocks.csv" },
    { "name": "seismic", "location": "https://happy-research.s3.ap-southeast-1.amazonaws.com/seismic.csv" },
    { "name": "shuttle", "location": "https://happy-research.s3.ap-southeast-1.amazonaws.com/shuttle.csv" },
    { "name": "speech", "location": "https://happy-research.s3.ap-southeast-1.amazonaws.com/speech.csv" },
    { "name": "synthetic", "location": "https://happy-research.s3.ap-southeast-1.amazonaws.com/synthetic.csv" },
    { "name": "waveform", "location": "https://happy-research.s3.ap-southeast-1.amazonaws.com/waveform.csv" }
]

scores = [["Dataset", "Relative Frequency", "Semantic Variation"]]

for o in datasets:
    # Instantiate pandas DataFrame
    data = pd.DataFrame()

    # Chunk size for reading data
    chunksize = 10000

    # The reference to the dataset. Change this to 
    dataset_file = o["location"]

    print("Loading dataset '{}'...".format(dataset_file))

    # Read each chunk and append to data frame
    for i, chunk in enumerate(pd.read_csv(dataset_file, header=None, chunksize=chunksize)):
        print("Reading chunk %d" % (i + 1))
        data = data.append(chunk)

    print("Done loading dataset %s..." % (o["name"]))
    
    score_rf = score_relative_frequency(data)
    score_sv = score_semantic_variation(data)
    
    scores.append([
        o["name"],
        score_rf,
        score_sv
    ])

Loading dataset 'https://happy-research.s3.ap-southeast-1.amazonaws.com/annthyroid.csv'...
Reading chunk 1
Done loading dataset annthytoid...
Loading dataset 'https://happy-research.s3.ap-southeast-1.amazonaws.com/backdoor.csv'...
Reading chunk 1
Reading chunk 2
Reading chunk 3
Reading chunk 4
Reading chunk 5
Reading chunk 6
Reading chunk 7
Reading chunk 8
Reading chunk 9
Reading chunk 10
Done loading dataset backdoor...
Loading dataset 'https://happy-research.s3.ap-southeast-1.amazonaws.com/bald.csv'...
Reading chunk 1
Reading chunk 2
Reading chunk 3
Reading chunk 4
Reading chunk 5
Reading chunk 6
Reading chunk 7
Reading chunk 8
Reading chunk 9
Reading chunk 10
Reading chunk 11
Reading chunk 12
Reading chunk 13
Reading chunk 14
Reading chunk 15
Reading chunk 16
Reading chunk 17
Reading chunk 18
Reading chunk 19
Reading chunk 20
Reading chunk 21
Done loading dataset bald...
Loading dataset 'https://happy-research.s3.ap-southeast-1.amazonaws.com/bank.csv'...
Reading chunk 1
Reading chun

In [6]:
# Display result in tabular format
tabulate.tabulate(scores, tablefmt='html')

Dataset,Relative Frequency,Semantic Variation
annthytoid,7.416666666666667,0.204576148176165
backdoor,2.443118043827167,0.3820243613622607
bald,2.244334868385333,0.20706683197942483
bank,11.265417111780131,-0.0702057753541661
cover,0.9603283365029646,0.5323430239198219
creditcardfraud,0.1727485630620034,-1.816141037269964
donors,5.927411411760527,1.2317944589464498
kddcup99,0.4074081772641294,-1.5189374087035048
magic04,35.16298633017876,-0.46289559058591584
mammography,2.3249575248144505,0.4204377120843123
